# Target 값 추출

Target 값
 - Period : n일 후 목표 변동률에 도달한다. 
 - pred10EMA : 10분후의 EMA 값을 목표로 한다. 

In [1]:
import numpy as np
import pandas as pd
import ctypes

from modules.local_values import raw_datas,datas
import modules.indicators as idc 
pd.set_option('float_format', '{:.2f}'.format)

In [2]:
### Period
from modules.c_dll import pred_close

df = pd.read_csv(raw_datas[0], parse_dates=[0], index_col=[0])
df=df.resample(rule='T').first()
df[['volume','value']]=df[['volume','value']].fillna(0)
df=df.interpolate()
c_y = pred_close(0.005, df['close'])
y = pd.DataFrame(list(c_y),columns=['period'])
y.index = df.index

In [3]:
df['pred20EMA10'] = idc.EMA(df,10).shift(-20)-df.close
y['pred20EMA10'] = df['pred20EMA10'] / df['pred20EMA10'].abs().max()

In [4]:
len(df)

3445152

## 정규화 및 값 저장

In [5]:
from modules.local_values import raw_datas, target_datas

for file,raw_file in zip(target_datas,raw_datas):
    df=pd.read_csv(raw_file,parse_dates=[0],index_col=[0])
    df=df.resample(rule='T').first()
    df[['volume','value']]=df[['volume','value']].fillna(0)
    df=df.interpolate()s
    
    c_y = pred_close(0.005, df['close'])
    y = pd.DataFrame(list(c_y),columns=['period'])
    y.index = df.index

    df['pred20EMA10'] = idc.EMA(df,10).shift(-20)-df.close
    y['pred20EMA10'] = df['pred20EMA10'] / df['pred20EMA10'].abs().max()

    y = (y - y.mean())/y.std()

    y.to_csv(file)